In [28]:
import os
import pandas as pd

# Define the path to the folder
path = 'RMMS/'

# Get a list of all files in the folder
files = [f for f in os.listdir(path) if f.endswith('.lrps.htm')]

# Initialize an empty list to store DataFrames
dfs = []

# Loop through each file
for file in files:
    # Read HTML tables into a list of DataFrames
    tables = pd.read_html(os.path.join(path, file))
    
    # Combine all tables into a single DataFrame
    df = pd.concat(tables)
    
    # Remove the first row and rows above row 9
    df = df.iloc[9:].reset_index(drop=True)
    
    # Set the correct header using the values from row 9
    df.columns = df.iloc[0]
    
    # Drop the now redundant row 9
    df = df.iloc[1:].reset_index(drop=True)
    
    # Drop the first and last columns
    df = df.iloc[:, 1:-1]
    
    # Add a new column for road number
    road_number = file.split('.')[0]  # Extract road number from file name
    df['road_number'] = road_number
    
    # Append the DataFrame to the list
    dfs.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dfs, ignore_index=True)

# Save the combined DataFrame to a CSV file
combined_df.to_csv('combined_lrps.csv', index=False)


FileNotFoundError: [WinError 3] Het systeem kan het opgegeven pad niet vinden: 'RMMS/'

In [27]:
combined_df.shape
combined_df.to_csv('RMMS/combined_lrps.csv', index=False)

NameError: name 'combined_df' is not defined

In [9]:
import pandas as pd

traffic_df = pd.read_csv('traffic.csv')
traffic_df.head()

,road,name,LRP,Offset,Chainage,LRP.1,Offset.1,Chainage.1,(Km),Heavy Truck,...,Car,Auto Rickshaw,Motor Cycle,Bi-Cycle,Cycle Rickshaw,Cart,Motorized,Non Motorized,Total AADT,(AADT)
0,N1,Jatrabari - Int.with Z1101 (Left) (Left),LRPS,0,0.000,LRPS,822,0.822,0.822,402.0,...,1851.0,2980.0,398.0,232.0,889.0,0.0,18236.0,1121.0,19357.0,19357.0
1,N1,Jatrabari - Int.with Z1101 (Left) (Right),LRPS,0,0.000,LRPS,822,0.822,0.822,660.0,...,2608.0,2508.0,436.0,213.0,1088.0,0.0,20236.0,1301.0,21537.0,21537.0
2,N1,Int.with Z1101 - Signboard (Left) R111 (Left),LRPS,822,0.822,LRPS,4175,4.175,3.353,660.0,...,2608.0,2508.0,436.0,213.0,1088.0,0.0,20236.0,1301.0,21537.0,21537.0
3,N1,Int.with Z1101 - Signboard (Left) R111 (Right),LRPS,822,0.822,LRPS,4175,4.175,3.353,402.0,...,1851.0,2980.0,398.0,232.0,889.0,0.0,18236.0,1121.0,19357.0,19357.0
4,N1,Signboard - Shimrail (Left)R110 (Left),LRPS,4175,4.175,LRPS,7181,7.181,3.006,91.0,...,1690.0,2266.0,1087.0,75.0,1198.0,0.0,16288.0,1273.0,17561.0,17561.0


In [56]:
traffic_df= traffic_df.rename(columns = { 'LRP':'LRP Start', 'Offset': 'Offset Start', 'Chainage':'Chainage Start', 'LRP.1':'LRP End', 'Offset.1': 'Offset End', 'Chainage.1':'Chainage End'})
print(traffic_df.shape)


(2066, 26)


In [21]:
# Combining the different dataframes into one so we have the lat/lon values combined with the AADT values so we can calculate the different economic values for the road parts.  
combined_lrps = pd.read_csv('combined_lrps.csv')

latlonload = pd.merge(traffic_df, combined_lrps, left_on=['LRP Start','road'], right_on=['LRP No', 'road_number'] )

In [31]:
#When merging we got 3 rows of each entry, we have dropped the duplicates below to ensure we do not get double entries.
latlonload = latlonload.drop_duplicates()
latlonload.to_csv('latlonloads.csv')

In [61]:
#To calculate the economic value of a roadpart, we have to assign economic value to different vehicle types. Afterwards we will multiply these values with the amount of sightings on each road to compute the economic value of a road. The logic behind the values is that for example trucks are able to carry a lot of goods, busses a lot of people and smaller vehicles are more individual focussed, hence the smaller economic impact. It should be noted that these are all estimates as we were unable to find research on the marginal economic impact of these vehicle types. 

# Define economic value vehicles dictionary
economic_value_vehicles = {'Heavy Truck':10, 'Medium Truck': 8, 'Small Truck':7, 'Large Bus':7, 'Medium Bus':6,
                           'Micro Bus':5, 'Utility':3, 'Car':3, 'Auto Rickshaw':2, 'Motor Cycle': 1,
                           'Bi-Cycle':0.5, 'Cycle Rickshaw':1, 'Cart':1}

# Define a function to check if a value can be converted to float
def is_float(x):
    try:
        float(x)
        return True
    except ValueError:
        return False

# Apply the function to each element of the DataFrame
for col in economic_df.columns:
    non_numeric_values = economic_df[col][~economic_df[col].apply(is_float)]
    if not non_numeric_values.empty:
        print(f"Non-numeric values in column '{col}':")
        print(non_numeric_values)

# Drop rows containing 'NS' values because there were rows with only NS values where floats were supposed to be.
latlonload = latlonload[~latlonload.apply(lambda row: (row == 'NS').any(), axis=1)]



In [62]:
# Select columns relevant for economic calculation
economic_df = latlonload.loc[:, 'Heavy Truck':'Cart']

# Convert values to floats
economic_df = economic_df.astype(float)

# Calculate the economic value for each row
latlonload['EVV'] = economic_df.apply(lambda row: sum(row[col] * economic_value_vehicles[col] for col in economic_df.columns if col in economic_value_vehicles), axis=1)

latlonload.tail(10)

,road,name,LRP Start,Offset Start,Chainage Start,LRP End,Offset End,Chainage End,(Km),Heavy Truck,...,Total AADT,(AADT),LRP No,Road Chainage,LRP TYPE,Description,Latitude Decimal,Longitued Decimal,road_number,EVV
6162,Z8909,Char Fession (Int.with R890) - Betua,LRPS,0,0.000,LRP006,952,7.002,7.002,0.0,...,3991.0,3991.0,LRPS,0,Others,Start from R890 at Thana road more char fassion,22.185833,90.76025,Z8909,5192.5
6165,Z8913,Debirchar - Lalmohon (Int.with R890),LRPS,0,0.000,LRPS,11325,11.325,11.325,24.0,...,6344.0,6344.0,LRPS,0,Others,"Start of debir char high school , Lalmohon Bhola",22.396083,90.6886663,Z8913,10778.0
6168,Z8913,Lalmohon (Int.with R890) - Tazumuddin (Int.wit...,LRPS,11325,11.325,LRPS,26505,26.505,15.180,0.0,...,2418.0,2418.0,LRPS,0,Others,"Start of debir char high school , Lalmohon Bhola",22.396083,90.6886663,Z8913,3108.0
6171,Z8915,Bagmara - Bangla Bazar (Int.with R890),LRPS,0,0.000,LRP004,1020,4.350,4.350,0.0,...,3522.0,3522.0,LRPS,0,Others,Start at Bagmara Infront of Launch Ghat.,22.5893886,90.6194716,Z8915,7911.0
6174,Z8915,Bangla Bazar (Int.with R890) - Int.with Z8916,LRP004,1020,4.350,LRP010,1150,10.470,6.120,0.0,...,3117.0,3117.0,LRP004,3.366,Km Post,Inf missing,22.5926941,90.6494719,Z8915,6379.0
6177,Z8915,Int.with Z8916 - Daulatkhan,LRP010,1150,10.470,LRP014,2290,15.660,5.190,0.0,...,3117.0,3117.0,LRP010,9.376,Km Post,Inf missing,22.6011389,90.7048333,Z8915,6379.0
6180,Z8916,Guingerhat (Int.with R890) - Int.with Z8915,LRPS,0,0.000,LRP004,3626,7.626,7.626,0.0,...,2440.0,2440.0,LRPS,0,Others,Start from R890 at Guingerhat Bazar Dhola,22.6254993,90.6617222,Z8916,3553.5
6183,Z8916,Int.with Z8915 - Daulatkhan Bazar,LRP004,3626,7.626,LRP005,6600,11.591,3.965,0.0,...,2440.0,2440.0,LRP004,4,Km Post,Information missing,22.6189716,90.6941108,Z8916,3553.5
6186,Z8943,Tazumuddin (Int.with Z8905) - Fakirhat (Int.wi...,LRPS,0,0.000,LRPS,6446,6.446,6.446,0.0,...,3083.0,3083.0,LRPS,0,Others,Start at intersection of Z8905 /BC joint infro...,22.4264438,90.8494719,Z8943,5651.0
6189,Z8943,Fakirhat (Int.with Z8948) - Kungerhat (Int.wit...,LRPS,6446,6.446,LRPS,8800,8.800,2.354,0.0,...,3083.0,3083.0,LRPS,0,Others,Start at intersection of Z8905 /BC joint infro...,22.4264438,90.8494719,Z8943,5651.0


In [65]:
latlonload.sort_values('EVV', ascending=False)

,road,name,LRP Start,Offset Start,Chainage Start,LRP End,Offset End,Chainage End,(Km),Heavy Truck,...,Total AADT,(AADT),LRP No,Road Chainage,LRP TYPE,Description,Latitude Decimal,Longitued Decimal,road_number,EVV
651,N3,Banani Rail Crossing-Progoti sharani(Left) (Left),LRPS,0,0.000,LRP001a,0,1.810,1.810,45.0,...,82414.0,82414.0,LRPS,0,Others,Start of Road From the End of Banani Rail Cros...,23.8145833,90.4043056,N3,311371.0
654,N3,Banani Rail Crossing-Progoti sharani(Left) (Ri...,LRPS,0,0.000,LRP001a,0,1.810,1.810,127.0,...,81610.0,81610.0,LRPS,0,Others,Start of Road From the End of Banani Rail Cros...,23.8145833,90.4043056,N3,309359.0
657,N3,"Progoti sharani-Abdullahpur, Int. with N302 (L...",LRP001a,0,1.810,LRP001a,7247,9.057,7.247,38.0,...,73476.0,73476.0,LRP001a,1.81,"Side Road, Right",Progoti Sharani,23.8217771,90.4191104,N3,284732.0
660,N3,"Progoti sharani-Abdullahpur, Int. with N302 (L...",LRP001a,0,1.810,LRP001a,7247,9.057,7.247,234.0,...,69068.0,69068.0,LRP001a,1.81,"Side Road, Right",Progoti Sharani,23.8217771,90.4191104,N3,278333.0
810,N4,"Karotia(Int.with Z4012)-Tangail, Nagarjalphai ...",LRP056,1815,55.708,LRP056,3788,57.681,1.973,118.0,...,27674.0,27674.0,LRP056,54.309,Km Post,Km post missing,24.2210549,89.9674993,N4,156204.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5280,Z6855,Bagshara-Chabbis Nagar (int.with Z6809),LRPS,0,0.000,LRP014,999,15.479,15.479,0.0,...,783.0,783.0,LRPS,0,Others,Start from bagshara more.,24.4647778,88.5961108,Z6855,594.5
4773,Z5408,Kadda-Tamai,LRPS,0,0.000,LRPS,7890,7.890,7.890,0.0,...,584.0,584.0,LRPS,0,Others,"Start of road at Jamtail, Bottola 21 m East fr...",24.3649444,89.6626389,Z5408,522.0
3834,Z1612,Bogachhari-Langadu,LRPS,0,0.000,LRP008,418,8.428,8.428,0.0,...,298.0,298.0,LRPS,0,Others,Road Start from R 162 at Bogachhari.,22.8130278,92.0384441,Z1612,495.5
3459,Z1084,Muhuriganj-Int.with Z1025,LRPS,0,0.000,LRPS,11000,11.000,11.000,0.0,...,228.0,228.0,LRPS,0,Others,Start of Road From Jingira,22.8594722,91.5298049,Z1084,271.0
